In [1]:
from Acesso import Login
from Query import  Query
import pandas as pd

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
querys={
    
    'Vendas':
    
    """
    
    DECLARE @DTBASE DATETIME,@DTFIM DATETIME,@DTINICIO DATETIME,@DIAS SMALLINT

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS date),101)

    SET @DIAS=DAY(@DTBASE)

    SET @DTFIM=@DTBASE

    SET @DTINICIO=DATEADD(MONTH,-1,DATEADD(DAY,1,DATEADD(DAY,@DIAS*-1,@DTFIM)))

    SELECT * FROM netfeira.vw_venda_estatico
    WHERE [Data de Faturamento] BETWEEN @DTINICIO AND @DTFIM AND [Tipo de Operação]='VENDAS'
    ORDER BY [Data de Faturamento]  
    
    """,
    
    'Meta':
    
    """
    
    SELECT * FROM netfeira.vw_metas
    WHERE [Meta R$]>0
    
    """,
    
    'Vendedor':
    
    """

    SELECT v.[ID Vendedor],v.[Nome Resumido],s.Equipe,s.[ID Sup],s.Supervisor,s.[DDD Sup],s.[Telefone Sup],
    s.[ID Gerente],s.Gerente,s.[DDD Gerente],s.[Telefone Gerente]
    FROM netfeira.vw_vendedor v
    INNER JOIN netfeira.vw_supervisor s ON v.[ID Equipe]=s.[ID Equipe]
    WHERE v.[Status do Vendedor]='ATIVO'

    """,
    
    'Cliente':
    
    """
    
    SELECT * FROM netfeira.vw_cliente
    
    """,
    
    'Calendario':
    
    """
    
    DECLARE @DTBASE DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS date),101)

    SELECT COUNT(Data) AS [Útil],COUNT([Data Trabalhada])-1 AS [Trabalhado],COUNT(Data)-(COUNT([Data Trabalhada])-1) AS [Restante]
    FROM netfeira.vw_calend
    WHERE YEAR(Data)=YEAR(@DTBASE) AND MONTH(Data)=MONTH(@DTBASE) AND [Dia Útil]=1    

    """
    
}

In [4]:
s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

In [5]:
df=sql.CriarTabela(kwargs=querys)

C:\Users\TI\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\TI\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\TI\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\TI\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 c

In [6]:
df['Vendas'].columns

Index(['ID Empresa', 'Pedido', 'NFe', 'Tipo de Pedido', 'Tipo de Operação',
       'Tabelas', 'Origem', 'Situação', 'Data de Emissão',
       'Data de Faturamento', 'ID Cliente', 'ID Vendedor', 'SKU', 'Seq',
       'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor de VDA', 'Total Geral',
       'Total Venda', 'Comissão R$', 'COFINS R$', 'PIS R$', 'ICMS R$',
       'ICMS ST R$', 'Peso Bruto KG', 'Peso Líquido KG', 'Margem Bruta R$'],
      dtype='object')

In [7]:
df['Vendedor'].columns

Index(['ID Vendedor', 'Nome Resumido', 'Equipe', 'ID Sup', 'Supervisor',
       'DDD Sup', 'Telefone Sup', 'ID Gerente', 'Gerente', 'DDD Gerente',
       'Telefone Gerente'],
      dtype='object')

In [8]:
df['Meta'].columns

Index(['ID Vendedor', 'Meta R$'], dtype='object')

In [9]:
df['Cliente'].columns

Index(['ID Cliente', 'CNPJ', 'Rede', 'CNPJ Base', 'CNPJ Caracter',
       'Razão Social', 'Nome Fantasia', 'Tipo de Cliente', 'Status do Cliente',
       'ID Segmento', 'Matriz', 'Crédito', 'Data de Cadastro',
       'Primeira Compra', 'Última Compra', 'Dias Compra', 'Tabela',
       'Condição de Pagto', 'Prazo Pagto', 'Pagamento', 'CEP', 'Endereço',
       'Bairro', 'Município', 'Numero', 'Complemento', 'Região', 'UF', 'DDD',
       'Contato', 'Limite de Crédito', 'Principal', 'E-mail Cliente',
       'Tributação do Cliente', 'ID Rota', 'Latitude', 'Longitude',
       'Desc Financeiro'],
      dtype='object')

In [10]:
df['Calendario'].columns

Index(['Útil', 'Trabalhado', 'Restante'], dtype='object')

In [11]:
from datetime import datetime

In [12]:
dt_atu=datetime.now().date()

dia_util=df['Calendario']['Útil'].values[-1]

dia_trab=df['Calendario']['Trabalhado'].values[-1]

dia_restante=df['Calendario']['Restante'].values[-1]

In [13]:
#criar base de dados

df['Temp']=df['Vendas'].loc[(df['Vendas']['Data de Faturamento'].dt.year==dt_atu.year)&(df['Vendas']['Data de Faturamento'].dt.month==dt_atu.month)]

df['Temp']=df['Temp'].groupby(['ID Vendedor'],as_index=False).agg({'Total Venda':'sum'})

df['Temp']=df['Temp'].merge(df['Meta'],on='ID Vendedor',how='inner')

df['Temp']=df['Temp'].merge(df['Vendedor'],on='ID Vendedor',how='inner')

df['Temp']=df['Temp'].groupby(['Equipe'],as_index=False).agg({'Total Venda':'sum','Meta R$':'sum'})

df['Temp']['Projeção']=round((df['Temp']['Total Venda']/dia_trab)*dia_util,2)

df['Temp']['Status']=df['Temp'].apply(lambda info: 1 if info['Projeção']>=info['Meta R$'] else 0,axis=1)

df['Temp']=df['Temp'].loc[df['Temp']['Status']==0]

equipes=df['Temp']['Equipe'].tolist()

codigos=df['Vendedor']['ID Vendedor'].loc[df['Vendedor']['Equipe'].isin(equipes)].tolist()

ValueError: Expected a 1D array, got an array with shape (0, 4)

In [ ]:
#consolidado dados

df['Temp']=df['Vendas'].loc[(df['Vendas']['Data de Faturamento'].dt.year==dt_atu.year)&(df['Vendas']['Data de Faturamento'].dt.month==dt_atu.month)&(df['Vendas']['ID Vendedor'].isin(codigos))]

df['Temp']=df['Temp'].groupby(['ID Vendedor'],as_index=False).agg({'Total Venda':'sum'})

df['Temp']=df['Temp'].merge(df['Meta'],on='ID Vendedor',how='inner')

df['Temp']=df['Temp'].merge(df['Vendedor'],on='ID Vendedor',how='inner')[['ID Vendedor', 'Nome Resumido', 'Equipe', 'ID Sup', 'Supervisor',
       'DDD Sup', 'Telefone Sup', 'ID Gerente', 'Gerente', 'DDD Gerente',
       'Telefone Gerente','Total Venda','Meta R$']]

df['Temp']['Projeção']=round((df['Temp']['Total Venda']/dia_trab)*dia_util,2)

df['Temp']['Status']=df['Temp'].apply(lambda info: 1 if info['Projeção']>=info['Meta R$'] else 0,axis=1)

df['Temp']=df['Temp'].loc[df['Temp']['Status']==0]

df['Temp'].sort_values('Total Venda',ascending=False,inplace=True)

codigos=df['Temp']['ID Vendedor'].tolist()

In [ ]:
valores=[]

diferenca=[]

for c in codigos:
    
    df['Diario']=df['Vendas'].loc[(df['Vendas']['Data de Faturamento'].dt.year==dt_atu.year)&(df['Vendas']['Data de Faturamento'].dt.month==dt_atu.month)&(df['Vendas']['ID Vendedor']==c)]
    
    meta=df['Temp']['Meta R$'].loc[df['Temp']['ID Vendedor']==c].values[-1]
    
    meta=round(meta/dia_util,2)
    
    df['Diario']=df['Diario'].groupby(['Data de Faturamento'],as_index=False).agg({'Total Venda':'sum'})
    
    df['Diario'].sort_values('Data de Faturamento',ascending=True,inplace=True)
    
    df['Diario']['Meta']=meta
    
    df['Diario']['Dif']=round(df['Diario']['Total Venda']-df['Diario']['Meta'],2)
        
    dif=df['Diario']['Dif'].sum()
    
    total=round(abs(dif)+meta if dif<0 else meta-dif,2)
    
    diferenca.append(dif)
    
    valores.append(total)
    
    pass

df['Temp']['Meta Diária']=round(df['Temp']['Meta R$']/dia_util,2)

df['Temp']['Débito Diário']=diferenca

df['Temp']['Á Realizar']=valores

df['Temp']['Á Realizar']=round(df['Temp']['Á Realizar']/dia_restante,2)

df['Temp'].sort_values('Débito Diário',ascending=True,inplace=True,ignore_index=True)


In [ ]:
df['Temp'].loc[df['Temp']['Débito Diário']>0,'Meta Diária']=df['Temp'].loc[df['Temp']['Débito Diário']>0].apply(lambda info: round((info['Meta R$']-info['Total Venda'])/dia_restante,2),axis=1)

In [ ]:
df['Temp'].loc[df['Temp']['Débito Diário']>0,'Á Realizar']=df['Temp'].loc[df['Temp']['Débito Diário']>0,'Meta Diária']

In [ ]:
codigos=df['Temp']['ID Vendedor'].tolist()

In [ ]:
df['Vendas']['Mês']=df['Vendas']['Data de Faturamento'].dt.month

In [ ]:
#relação de clientes

df['Lista']=df['Vendas'].loc[df['Vendas']['ID Vendedor'].isin(codigos)].groupby(['ID Cliente','ID Vendedor','Mês'],as_index=False).agg({'Total Venda':'sum'})

df['Lista']=df['Lista'].pivot(index=['ID Cliente','ID Vendedor'],columns='Mês',values='Total Venda').reset_index()

colunas=[l for l in df['Lista'].columns if str(l).isnumeric()]

meses={1:'JAN',2:'FEV',3:'MAR',4:'ABR',5:'MAI',6:'JUN',7:'JUL',8:'AGOS',9:'SET',10:'OUT',11:'NOV',12:'DEZ'}

for i,c in enumerate(colunas):
    
    if(i==0):
        
        df['Lista']=df['Lista'].loc[df['Lista'][c]>0]
        
        pass
    
    df['Lista'].loc[df['Lista'][c].isnull(),c]=0
    
    df['Lista'].rename(columns={c:meses[c]},inplace=True)
    
    pass

df['Lista'].loc[df['Lista'][df['Lista'].columns[-1]]<=0,'Positivado']='N'

df['Lista'].loc[df['Lista']['Positivado'].isnull(),'Positivado']='S'

df['Lista']=df['Cliente'].merge(df['Lista'],on='ID Cliente',how='inner')

df['Lista']=df['Vendedor'].merge(df['Lista'],on='ID Vendedor',how='inner')

df['Lista']


In [ ]:
colunas={'ID Sup':'ID Gerente','ID Gerente':'ID Gerente'}

nomes={'ID Sup':'Supervisor','ID Gerente':'Gerente'}

tel_ddd={'ID Sup':'DDD Sup','ID Gerente':'DDD Gerente'}

tel_num={'ID Sup':'Telefone Sup','ID Gerente':'Telefone Gerente'}

for col1,col2 in colunas.items():
    
    codigos=df['Temp'][col1].unique().tolist()
    
    for c in codigos:
        
        cod=df['Temp'][col2].loc[df['Temp'][col1]==c].unique().tolist()[-1]
        
        if(c==cod and col1=='ID Sup'):
            
            continue
                
        display(df['Lista'].loc[df['Lista'][col1]==c])
        
        pass
    
    pass
